In [56]:
from dashboard.tabs.riboseq_atlas import get_average_coverage


In [37]:
#     from dashboard import plotting, description
#     from dashboard.etl.sorf_query import load_jsonlines_table
    
#     import gzip
#     import shlex
    
#     import pathlib
#     import subprocess
#     import pandas as pd
#     import click
    
#     input_vtx = pathlib.Path('/home/ec2-user/repos/dashboard/cache/protein_data/protein_tools_input.fasta')
#     output_prefix = pathlib.Path('/home/ec2-user/repos/dashboard/cache/protein_data')

# # input_vtx = #pathlib.Path('/home/ec2-user/repos/dashboard/cache/protein_data/protein_tools_input.fasta')
# # CACHE_DIR = pathlib.Path('/home/ec2-user/repos/dashboard/cache')
# # output_prefix = pathlib.Path('/home/ec2-user/repos/dashboard/cache/protein_data')

def run_protein_search_tools(input_vtx, output_prefix):
    input_vtx = pathlib.Path(input_vtx)
    output_prefix = pathlib.Path(output_prefix)
    
    REFSEQ_FASTA_FILE = output_prefix.joinpath('GRCh38_latest_protein.faa')
    ENSEMBL_FASTA_FILE = output_prefix.joinpath('swissprot_proteins.fa')
    SWISSPROT_FASTA_FILE = output_prefix.joinpath('ensembl_proteins.fa')
    VTX_WITHOUT_SIGNAL_SEQ = output_prefix.joinpath('vtx_aa_seq_signal_sequence_removed.fa')
    any_missing = []
    for f in [REFSEQ_FASTA_FILE, ENSEMBL_FASTA_FILE, SWISSPROT_FASTA_FILE, VTX_WITHOUT_SIGNAL_SEQ]:
        if not f.exists():
            any_missing.append(f)
    if len(any_missing) > 0:
        raise ValueError(f"Reference files are missing the following files don't exist. \
            please check that update_cache.py has completed, or create these manually in the output_prefix directory.: {any_missing}")
    # Swissprot Isoform Check
    fasta_file = SWISSPROT_FASTA_FILE
    query_db = f'/root/{input_vtx.name}'
    target_db = f'/root/{fasta_file.name}'
    output_file = f'/root/{input_vtx.stem}_{fasta_file.stem}_alignments.m8'
    options = '--format-output query,target,evalue,qstart,qend,qlen,qcov,gapopen,pident,fident,alnlen,raw,bits'

    base_cmd = f"docker run --rm -v {output_prefix}:/root soedinglab/mmseqs2 mmseqs easy-search" 
    full_cmd = f'{base_cmd} {options} {query_db} {target_db} {output_file} /root/tmp'
    subprocess.run(shlex.split(full_cmd))

    isoform_df = pd.read_csv(output_prefix.joinpath(f'{input_vtx.stem}_{fasta_file.stem}_alignments.m8'), sep='\t', names=options.split()[1].split(','))
    isoform_df = isoform_df[(isoform_df['pident'] == 100.0)]
    isoform_df.drop_duplicates(inplace=True)
    swissprot_isoform_df = isoform_df.groupby('query').aggregate(list)
    swissprot_isoform_df.rename(columns={'target': 'swissprot_isoform'}, inplace=True)

    # Ensembl Isoforms
    fasta_file = ENSEMBL_FASTA_FILE
    query_db = f'/root/{input_vtx.name}'
    target_db = f'/root/{fasta_file.name}'
    output_file = f'/root/{input_vtx.stem}_{fasta_file.stem}_alignments.m8'
    options = '--format-output query,target,evalue,qstart,qend,qlen,qcov,gapopen,pident,fident,alnlen,raw,bits'

    base_cmd = f"docker run --rm -v {output_prefix}:/root soedinglab/mmseqs2 mmseqs easy-search" 
    full_cmd = f'{base_cmd} {options} {query_db} {target_db} {output_file} /root/tmp'
    subprocess.run(shlex.split(full_cmd))
    
    isoform_df = pd.read_csv(output_prefix.joinpath(f'{input_vtx.stem}_{fasta_file.stem}_alignments.m8'), sep='\t', names=options.split()[1].split(','))
    isoform_df = isoform_df[(isoform_df['pident'] == 100.0)]
    isoform_df.drop_duplicates(inplace=True)
    ensembl_isoform_df = isoform_df.groupby('query').aggregate(list)
    ensembl_isoform_df.rename(columns={'target': 'ensembl_isoform'}, inplace=True)

    # Refseq Isoforms
    fasta_file = REFSEQ_FASTA_FILE
    query_db = f'/root/{input_vtx.name}'
    target_db = f'/root/{fasta_file.name}'
    output_file = f'/root/{input_vtx.stem}_{fasta_file.stem}_alignments.m8'
    options = '--format-output query,target,evalue,qstart,qend,qlen,qcov,gapopen,pident,fident,alnlen,raw,bits'

    base_cmd = f"docker run --rm -v {output_prefix}:/root soedinglab/mmseqs2 mmseqs easy-search" 
    full_cmd = f'{base_cmd} {options} {query_db} {target_db} {output_file} /root/tmp'
    subprocess.run(shlex.split(full_cmd))
    
    isoform_df = pd.read_csv(output_prefix.joinpath(f'{input_vtx.stem}_{fasta_file.stem}_alignments.m8'), sep='\t', names=options.split()[1].split(','))
    isoform_df = isoform_df[(isoform_df['pident'] == 100.0)]
    isoform_df.drop_duplicates(inplace=True)

    refseq_isoform_df = isoform_df.groupby('query').aggregate(list)
    refseq_isoform_df.rename(columns={'target': 'refseq_isoform'}, inplace=True)

#     swissprot_isoform_df.to_csv(pathlib.Path(output_prefix) / 'swissprot_isoform.csv')
#     ensembl_isoform_df.to_csv(pathlib.Path(output_prefix) / 'ensembl_isoform.csv')
#     refseq_isoform_df.to_csv(pathlib.Path(output_prefix) / 'refseq_isoform.csv')
    
    # Blast Tools
    header = ['vtx_id', 'blastp_hit_id', 'blastp_description', 'blastp_score',
          'blastp_query_coverage', 'blastp_align_length', 'blastp_align_identity', 
          'blastp_gaps', 'blastp_evalue']
    blast_db_path = pathlib.Path('/efs/databases/blast')
    blast_db = '-db mouse.protein.genbank.faa'
    output_fmt = '6 qaccver saccver stitle bitscore qcovs length pident gaps evalue'
    options = f'-outfmt "{output_fmt}" -num_threads 8'
    query = f'-query /blast/data/{input_vtx.name}'
    output = f'-out /blast/data/{input_vtx.stem}.blastp.out'
    base_cmd = f'docker run --rm -v {blast_db_path}:/blast/blastdb -v {output_prefix}:/blast/data ncbi/blast'
    full_cmd = f'{base_cmd} blastp {options} {blast_db} {query} {output}'
    subprocess.run(shlex.split(full_cmd))
    
    blastp_df = pd.read_csv(output_prefix.joinpath(f'{input_vtx.stem}.blastp.out'), sep='\t', names=header)
    bdf = blastp_df.sort_values(by='blastp_score', ascending=False).groupby('vtx_id').first()
    bdf.to_parquet(pathlib.Path(output_prefix).joinpath('mouse_blastp.parq'))

    header = ['vtx_id', 'tblastn_hit_id', 'tblastn_description', 'tblastn_score',
          'tblastn_query_coverage', 'tblastn_align_length', 'tblastn_align_identity', 
          'tblastn_gaps', 'tblastn_evalue']
    blast_db_path = pathlib.Path('/efs/databases/blast')
    blast_db = '-db mouse.rna.fna'
    output_fmt = '6 qaccver saccver stitle score qcovs length pident gaps evalue'
    options = f'-outfmt "{output_fmt}" -num_threads 8'
    query = f'-query /blast/data/{input_vtx.name}'
    output = f'-out /blast/data/{input_vtx.stem}.tblastn.out'
    
    base_cmd = f'docker run --rm -v {blast_db_path}:/blast/blastdb -v {output_prefix}:/blast/data ncbi/blast'
    full_cmd = f'{base_cmd} tblastn {options} {blast_db} {query} {output}'
    subprocess.run(shlex.split(full_cmd))
    

    tblastn_df = pd.read_csv(output_prefix.joinpath(f'{input_vtx.stem}.tblastn.out'), sep='\t', names=header)
    tdf = tblastn_df.sort_values(by='tblastn_score', ascending=False).groupby('vtx_id').first()
    tdf.to_parquet(pathlib.Path(output_prefix).joinpath('mouse_tblastn.parq'))
    
    subprocess.run(shlex.split(f'docker run  --rm -v {output_prefix}:/data -v /efs/databases/blast:/db ncbi/blast blastp -task blastp-fast -outfmt 15 -db /db/mouse.protein.faa -query /data/{input_vtx.stem} -max_target_seqs 20 -num_threads 8 -out /data/blastp.results.json'))
    
    # Merge isoform results
    isoform_df = pd.DataFrame([swissprot_isoform_df['swissprot_isoform'], ensembl_isoform_df['ensembl_isoform'], refseq_isoform_df['refseq_isoform']]).T
    # isoform_df.replace(pd.NA, 'None', inplace=True)
    isoform_df.to_parquet(pathlib.Path(output_prefix).joinpath('isoforms_search.parq'))
    
    # Run blast against sequneces with signal peptide removed
    #Blast Tools
    blast_db_path = pathlib.Path('/efs/databases/blast')
    blast_db = '-db mouse.protein.genbank.faa'
    output_fmt = '6 qaccver saccver stitle bitscore qcovs length pident gaps evalue'
    options = f'-outfmt "{output_fmt}" -num_threads 8'
    query = f'-query /blast/data/{VTX_WITHOUT_SIGNAL_SEQ.name}'
    output = f'-out /blast/data/{VTX_WITHOUT_SIGNAL_SEQ.stem}.blastp.out'
    base_cmd = f'docker run --rm -v {blast_db_path}:/blast/blastdb -v {output_prefix}:/blast/data ncbi/blast'
    full_cmd = f'{base_cmd} blastp {options} {blast_db} {query} {output}'
    subprocess.run(shlex.split(full_cmd))
    
    header = [
        'vtx_id', 'nonsig_blastp_refseq_id', 'nonsig_blastp_hit_id', 'nonsig_blastp_description', 'nonsig_blastp_score',
        'nonsig_blastp_align_length', 'nonsig_blastp_align_identity', 'nonsig_blastp_gaps', 'nonsig_blastp_evalue']
    
    non_sig_blastp_df = pd.read_csv(output_prefix.joinpath(f'{VTX_WITHOUT_SIGNAL_SEQ.stem}.blastp.out'), sep='\t', names=header)
    
    header = [
        'vtx_id', 'nonsig_tblastn_refseq_id', 'nonsig_tblastn_hit_id', 'nonsig_tblastn_description', 'nonsig_tblastn_score',
        'nonsig_tblastn_align_length', 'nonsig_tblastn_align_identity', 'nonsig_tblastn_gaps', 'nonsig_tblastn_evalue']
    output = f'-out /blast/data/{VTX_WITHOUT_SIGNAL_SEQ.stem}.tblastn.out'
    blast_db = '-db mouse.rna.fna'
    base_cmd = f'docker run --rm -v {blast_db_path}:/blast/blastdb -v {output_prefix}:/blast/data ncbi/blast'
    full_cmd = f'{base_cmd} tblastn {options} {blast_db} {query} {output}'
    subprocess.run(shlex.split(full_cmd))
    tblastn_df = pd.read_csv(output_prefix.joinpath(f'{VTX_WITHOUT_SIGNAL_SEQ.stem}.tblastn.out'), sep='\t', names=header)
    tdf = tblastn_df.sort_values(by='nonsig_tblastn_score', ascending=False).groupby('vtx_id').first()
    bdf = non_sig_blastp_df.sort_values('nonsig_blastp_score', ascending=False).groupby('vtx_id').first()
    output_nonsig_df = bdf.merge(tdf, left_index=True, right_index=True, how='outer')
    output_nonsig_df.to_parquet(output_prefix.joinpath('nonsignal_seq_blast_tblastn.parq'))

run_protein_search_tools(input_vtx, output_prefix)

/root/protein_tools_input_ensembl_proteins_alignments.m8 exists and will be overwritten
easy-search --format-output query,target,evalue,qstart,qend,qlen,qcov,gapopen,pident,fident,alnlen,raw,bits /root/protein_tools_input.fasta /root/ensembl_proteins.fa /root/protein_tools_input_ensembl_proteins_alignments.m8 /root/tmp 

MMseqs Version:                        	ad5837b3444728411e6c90f8c6ba9370f665c443
Substitution matrix                    	nucl:nucleotide.out,aa:blosum62.out
Add backtrace                          	false
Alignment mode                         	3
Alignment mode                         	0
Allow wrapped scoring                  	false
E-value threshold                      	0.001
Seq. id. threshold                     	0
Min alignment length                   	0
Seq. id. mode                          	0
Alternative alignments                 	0
Coverage threshold                     	0
Coverage mode                          	0
Max sequence length                    	65535


Command line argument error: Argument "query". File is not accessible:  `/data/protein_tools_input'


In [53]:
    header = ['vtx_id', 'blastp_hit_id', 'blastp_description', 'blastp_score',
          'blastp_query_coverage', 'blastp_align_length', 'blastp_align_identity', 
          'blastp_gaps', 'blastp_evalue']
    blast_db_path = pathlib.Path('/efs/databases/blast')
    blast_db = '-db mouse.protein.faa'
    output_fmt = '6 qaccver saccver stitle bitscore qcovs length pident gaps evalue'
    options = f'-outfmt "{output_fmt}" -num_threads 8'
    query = f'-query /blast/data/{input_vtx.name}'
    output = f'-out /blast/data/{input_vtx.stem}.blastp.out'
    base_cmd = f'docker run --rm -v {blast_db_path}:/blast/blastdb -v {output_prefix}:/blast/data ncbi/blast'
    full_cmd = f'{base_cmd} blastp -task blastp-fast {options} {blast_db} {query} {output}'
    subprocess.run(shlex.split(full_cmd))

CompletedProcess(args=['docker', 'run', '--rm', '-v', '/efs/databases/blast:/blast/blastdb', '-v', '/home/ec2-user/repos/dashboard/cache/protein_data:/blast/data', 'ncbi/blast', 'blastp', '-task', 'blastp-fast', '-outfmt', '6 qaccver saccver stitle bitscore qcovs length pident gaps evalue', '-num_threads', '8', '-db', 'mouse.protein.faa', '-query', '/blast/data/protein_tools_input.fasta', '-out', '/blast/data/protein_tools_input.blastp.out'], returncode=0)

In [54]:
    blastp_df = pd.read_csv(output_prefix.joinpath(f'{input_vtx.stem}.blastp.out'), sep='\t', names=header)
    bdf = blastp_df.sort_values(by='blastp_score', ascending=False).groupby('vtx_id').first()
    bdf.to_parquet(pathlib.Path(output_prefix).joinpath('mouse_blastp.parq'))

In [55]:
bdf = pd.read_parquet('/home/ec2-user/repos/dashboard/cache/protein_data/mouse_blastp.parq')
bdf.loc['VTX-0849999']

blastp_hit_id                                                                      XP_006515041.1
blastp_description       XP_006515041.1 phosphatidate phosphatase LPIN1 isoform X3 [Mus musculus]
blastp_score                                                                                 25.8
blastp_query_coverage                                                                          56
blastp_align_length                                                                            23
blastp_align_identity                                                                      52.174
blastp_gaps                                                                                     0
blastp_evalue                                                                                 3.9
Name: VTX-0849999, dtype: object

In [38]:
    blastp_df = pd.read_csv(output_prefix.joinpath(f'{input_vtx.stem}.blastp.out'), sep='\t', names=header)
    bdf = blastp_df.sort_values(by='blastp_score', ascending=False).groupby('vtx_id').first()
    bdf

KeyError: 'blastp_score'

In [33]:
# Run blast against sequneces with signal peptide removed
#Blast Tools
blast_db_path = pathlib.Path('/efs/databases/blast')
blast_db = '-db mouse.protein.genbank.faa'
output_fmt = '6 qaccver saccver stitle bitscore qcovs length pident gaps evalue'
options = f'-outfmt "{output_fmt}" -num_threads 8'
query = f'-query /blast/data/{VTX_WITHOUT_SIGNAL_SEQ.name}'
output = f'-out /blast/data/{VTX_WITHOUT_SIGNAL_SEQ.stem}.blastp.out'
base_cmd = f'docker run --rm -v {blast_db_path}:/blast/blastdb -v {output_prefix}:/blast/data ncbi/blast'
full_cmd = f'{base_cmd} blastp {options} {blast_db} {query} {output}'
# subprocess.run(shlex.split(full_cmd))

header = [
    'vtx_id', 'blastp_refseq_id', 'nonsig_blastp_hit_id', 'nonsig_blastp_description', 'nonsig_blastp_score',
    'nonsig_blastp_align_length', 'nonsig_blastp_align_identity', 'nonsig_blastp_gaps', 'nonsig_blastp_evalue']

non_sig_blastp_df = pd.read_csv(output_prefix.joinpath(f'{VTX_WITHOUT_SIGNAL_SEQ.stem}.blastp.out'), sep='\t', names=header)

header = [
    'vtx_id', 'tblastn_refseq_id', 'nonsig_tblastn_hit_id', 'nonsig_tblastn_description', 'nonsig_tblastn_score',
    'nonsig_tblastn_align_length', 'nonsig_tblastn_align_identity', 'nonsig_tblastn_gaps', 'nonsig_tblastn_evalue']
output = f'-out /blast/data/{VTX_WITHOUT_SIGNAL_SEQ.stem}.tblastn.out'
blast_db = '-db mouse.rna.fna'
base_cmd = f'docker run --rm -v {blast_db_path}:/blast/blastdb -v {output_prefix}:/blast/data ncbi/blast'
full_cmd = f'{base_cmd} tblastn {options} {blast_db} {query} {output}'
# subprocess.run(shlex.split(full_cmd))
tblastn_df = pd.read_csv(output_prefix.joinpath(f'{VTX_WITHOUT_SIGNAL_SEQ.stem}.tblastn.out'), sep='\t', names=header)
tdf = tblastn_df.sort_values(by='nonsig_tblastn_score', ascending=False).groupby('vtx_id').first()
bdf = non_sig_blastp_df.sort_values('nonsig_blastp_score', ascending=False).groupby('vtx_id').first()
output_nonsig_df = bdf.merge(tdf, left_index=True, right_index=True, how='outer')
output_nonsig_df.to_parquet(output_prefix.joinpath('nonsignal_seq_blast_tblastn.parq'))

In [34]:
output_nonsig_df

,blastp_refseq_id,nonsig_blastp_hit_id,nonsig_blastp_description,nonsig_blastp_score,nonsig_blastp_align_length,nonsig_blastp_align_identity,nonsig_blastp_gaps,nonsig_blastp_evalue,tblastn_refseq_id,nonsig_tblastn_hit_id,nonsig_tblastn_description,nonsig_tblastn_score,nonsig_tblastn_align_length,nonsig_tblastn_align_identity,nonsig_tblastn_gaps,nonsig_tblastn_evalue
vtx_id,,,,,,,,,,,,,,,,
VTX-0006555,AAF15261.1,AAF15261.1 Bv8 variant 3 precursor [Mus musculus],158.0,100.0,80.0,95.000,0.0,2.280000e-51,NM_015768.3,"NM_015768.3 Mus musculus prokineticin 2 (Prok2), transcript variant 1, mRNA",148.0,100.0,101.0,75.248,21.0,2.310000e-42
VTX-0006906,NP_001030077.1,NP_001030077.1 sperm-associated antigen 11B isoform 3 precursor [Mus musculus],47.0,93.0,52.0,48.077,2.0,7.230000e-08,NM_001034905.2,"NM_001034905.2 Mus musculus sperm associated antigen 11B (Spag11b), transcript variant 3, mRNA",47.4,93.0,52.0,48.077,2.0,4.110000e-07
VTX-0009065,EDL22895.1,EDL22895.1 sperm acrosome associated 4 [Mus musculus],136.0,86.0,89.0,73.034,1.0,5.510000e-42,NM_029806.2,"NM_029806.2 Mus musculus Ly6/Plaur domain containing 5 (Lypd5), mRNA",30.8,77.0,85.0,32.941,16.0,1.800000e+00
VTX-0009335,AAH31737.1,"AAH31737.1 Gene model 75, (NCBI) [Mus musculus]",143.0,95.0,115.0,53.043,1.0,5.350000e-44,NM_203508.2,"NM_203508.2 Mus musculus epididymal protein 3B (Eddm3b), mRNA",142.0,95.0,115.0,53.043,1.0,1.990000e-40
VTX-0009367,AAI47100.1,"AAI47100.1 Gene model 94, (NCBI) [Mus musculus]",84.0,100.0,66.0,63.636,3.0,2.910000e-22,NM_001033280.2,"NM_001033280.2 Mus musculus predicted gene 94 (Gm94), mRNA",56.2,62.0,39.0,64.103,0.0,3.090000e-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VTX-0860429,NP_001074752.1,NP_001074752.1 NALCN channel auxiliary factor 2 [Mus musculus],30.4,81.0,39.0,38.462,0.0,2.300000e-01,XM_036160193.1,"XM_036160193.1 PREDICTED: Mus musculus UDP-Gal:betaGlcNAc beta 1,3-galactosyltransferase, polypeptide 5 (B3galt5), transcript variant X4, mRNA",28.9,67.0,32.0,34.375,0.0,1.300000e+00
VTX-0860430,sp|A2BDP1.1|NALF2_MOUSE,sp|A2BDP1.1|NALF2_MOUSE RecName: Full=NALCN channel auxiliary factor 2; AltName: Full=Protein TED; AltName: Full=Transmembrane protein 28; AltName: Full=Transmembrane protein FAM155B,30.4,81.0,39.0,38.462,0.0,2.300000e-01,XM_036160191.1,"XM_036160191.1 PREDICTED: Mus musculus UDP-Gal:betaGlcNAc beta 1,3-galactosyltransferase, polypeptide 5 (B3galt5), transcript variant X1, mRNA",28.9,67.0,32.0,34.375,0.0,1.300000e+00
VTX-0860431,NP_001398496.1,NP_001398496.1 peptidyl-prolyl cis-trans isomerase FKBP14 isoform 3 [Mus musculus],28.1,66.0,19.0,52.632,0.0,7.300000e-01,NM_172428.2,"NM_172428.2 Mus musculus coiled-coil domain containing 134 (Ccdc134), transcript variant 1, mRNA",25.4,79.0,23.0,47.826,0.0,8.400000e+00


In [28]:
tblastn_df = pd.read_csv(output_prefix.joinpath(f'{VTX_WITHOUT_SIGNAL_SEQ.stem}.tblastn.out'), sep='\t', names=header)


FileNotFoundError: [Errno 2] No such file or directory: '/home/ec2-user/repos/dashboard/cache/protein_data/vtx_aa_seq_signal_sequence_removed.tblastn.out'

In [27]:
full_cmd

'docker run --rm -v /efs/databases/blast:/blast/blastdb -v /home/ec2-user/repos/dashboard/cache/protein_data:/blast/data ncbi/blast blastp -outfmt "6 qaccver saccver stitle bitscore qcovs length pident gaps evalue" -num_threads 8 -db mouse.protein.genbank.faa -query /blast/data/vtx_aa_seq_signal_sequence_removed.fa -out /blast/data/vtx_aa_seq_signal_sequence_removed.tblastp.out'

In [25]:
ls /home/ec2-user/repos/dashboard/cache/protein_data/

GRCh38_latest_protein.faa
TMRs.gff3
blastp.results.json
deepsig.results
deeptmhmm_results.md
ensembl_isoform.csv
ensembl_proteins.fa
esmfold.jsonlines
isoforms_search.parq
kibby.out
mouse_blastp.csv
predicted_topologies.3line
prediction_results.txt
processed_entries.fasta
protein_tools_input.blastp.out
protein_tools_input.fasta
protein_tools_input.tblastn.out
protein_tools_input_GRCh38_latest_protein_alignments.m8
protein_tools_input_ensembl_proteins_alignments.m8
protein_tools_input_swissprot_proteins_alignments.m8
refseq_isoform.csv
region_output.gff3
results_summary.signalp5
sequence_features_scores.csv
sequence_features_strings.csv
swissprot_isoform.csv
swissprot_proteins.fa
tblastn.csv
tblastn.results.csv
tmp/
vtx_aa_seq_signal_sequence_removed.blastp.out
vtx_aa_seq_signal_sequence_removed.fa
xena.db


In [24]:
full_cmd

'docker run --rm -v /efs/databases/blast:/blast/blastdb -v /home/ec2-user/repos/dashboard/cache/protein_data:/blast/data ncbi/blast blastp -outfmt "6 qaccver saccver stitle bitscore qcovs length pident gaps evalue" -num_threads 8 -db mouse.protein.genbank.faa -query /blast/data/vtx_aa_seq_signal_sequence_removed.fa -out /blast/data/vtx_aa_seq_signal_sequence_removed.blastp.out'

In [15]:
    isoform_df = pd.DataFrame([swissprot_isoform_df['swissprot_isoform'], ensembl_isoform_df['ensembl_isoform'], refseq_isoform_df['refseq_isoform']]).T
    # isoform_df.replace(pd.NA, ['None'], inplace=True)
    isoform_df.to_parquet(pathlib.Path(output_prefix).joinpath('isoforms_search.parq'))

In [47]:
import pandas as pd
from pathlib import Path
from ast import literal_eval

from dashboard.data_load import load_sorf_df_conformed, load_protein_feature_string_representations
from dashboard.etl import CACHE_DIR, DATA_DIR

In [54]:
df = load_sorf_df_conformed()

## BLASTp

In [7]:
with open('isoform_data/nonsignal_seq_aa.fa', 'w') as outfile:
    for i, row in df[df['nonsignal_seqs'] != ''].iterrows():
        outfile.write(f">{row.name}\n{row['nonsignal_seqs']}\n")

In [8]:
vtx_fasta = Path('isoform_data/nonsignal_seq_aa.fa').absolute()

In [12]:
data_path = Path('isoform_data').absolute()
blast_db_path = Path('/home/ubuntu/velia-data-dev/VDC_004_annotation/primary/blast/')
blast_db = '-db mouse.protein.genbank.faa'
output_fmt = '6 qaccver saccver stitle bitscore qcovs length pident gaps evalue'
options = f'-outfmt "{output_fmt}" -num_threads 8'
query = f'-query /blast/data/{vtx_fasta.name}'
output = f'-out /blast/data/{vtx_fasta.stem}.blastp.out'

base_cmd = f'docker run --rm -it -v {blast_db_path}:/blast/blastdb -v {data_path}:/blast/data ncbi/blast'
full_cmd = f'{base_cmd} blastp {options} {blast_db} {query} {output}'

In [14]:
!$full_cmd

In [15]:
header = ['vtx_id', 'blastp_hit_id', 'blastp_description', 'blastp_score',
          'blastp_query_coverage', 'blastp_align_length', 'blastp_align_identity', 
          'blastp_gaps', 'blastp_evalue']

blastp_df = pd.read_csv(f'isoform_data/{vtx_fasta.stem}.blastp.out', sep='\t', names=header)
bdf = blastp_df.sort_values(by='blastp_score', ascending=False).groupby('vtx_id').first()


## tBLASTn

In [17]:
data_path = Path('isoform_data').absolute()
blast_db_path = Path('/home/ubuntu/velia-data-dev/VDC_004_annotation/primary/blast/')
blast_db = '-db mouse.rna.fna'
output_fmt = '6 qaccver saccver stitle score qcovs length pident gaps evalue'
options = f'-outfmt "{output_fmt}" -num_threads 8'
query = f'-query /blast/data/{vtx_fasta.name}'
output = f'-out /blast/data/{vtx_fasta.stem}.tblastn.out'

base_cmd = f'docker run --rm -it -v {blast_db_path}:/blast/blastdb -v {data_path}:/blast/data ncbi/blast'
full_cmd = f'{base_cmd} tblastn {options} {blast_db} {query} {output}'

In [18]:
!$full_cmd

In [19]:
header = ['vtx_id', 'tblastn_hit_id', 'tblastn_description', 'tblastn_score',
          'tblastn_query_coverage', 'tblastn_align_length', 'tblastn_align_identity', 
          'tblastn_gaps', 'tblastn_evalue']

tblastn_df = pd.read_csv(f'isoform_data/{vtx_fasta.stem}.tblastn.out', sep='\t', names=header)
tdf = tblastn_df.sort_values(by='tblastn_score', ascending=False).groupby('vtx_id').first()

In [42]:
with open('../cache_updates/all_vtx_gencode_231218.txt', 'w') as outfile:
    for i, row in df.iterrows():
        outfile.write(f"{row.name}\n")



In [55]:
isoform_data_path = Path('/home/ubuntu/dashboard/scripts/notebooks/isoform_data/')
vtx_fasta = isoform_data_path.joinpath('nonsignal_seq_aa.fa')

header = [
    'vtx_id', 'blastp_refseq_id', 'nonsig_blastp_hit_id', 'nonsig_blastp_description', 'blastp_score',
    'nonsig_blastp_align_length', 'nonsig_blastp_align_identity', 'blastp_gaps', 'nonsig_blastp_evalue']

blastp_df = pd.read_csv(isoform_data_path.joinpath(f'{vtx_fasta.stem}.blastp.out'), sep='\t', names=header)
bdf = blastp_df.sort_values(by='blastp_score', ascending=False).groupby('vtx_id').first()

header = [
    'vtx_id', 'tblastn_refseq_id', 'nonsig_tblastn_hit_id', 'nonsig_tblastn_description', 'tblastn_score',
    'nonsig_tblastn_align_length', 'nonsig_tblastn_align_identity', 'tblastn_gaps', 'nonsig_tblastn_evalue']

tblastn_df = pd.read_csv(isoform_data_path.joinpath(f'{vtx_fasta.stem}.tblastn.out'), sep='\t', names=header)
tdf = tblastn_df.sort_values(by='tblastn_score', ascending=False).groupby('vtx_id').first()

df = df.merge(bdf, left_index=True, right_index=True, how='left')
df = df.merge(tdf, left_index=True, right_index=True, how='left')

#return df


In [58]:
df

,show_details,vtx_id,aa_length,ucsc_track,source,screening_phase_id,screening_phase,genscript_id,orf_xrefs,protein_xrefs,...,blastp_gaps,nonsig_blastp_evalue,tblastn_refseq_id,nonsig_tblastn_hit_id,nonsig_tblastn_description,tblastn_score,nonsig_tblastn_align_length,nonsig_tblastn_align_identity,tblastn_gaps,nonsig_tblastn_evalue
vtx_id,,,,,,,,,,,,,,,,,,,,,
VTX-0852017,False,VTX-0852017,54,chr1:-1--1,[velia_phase1_Bona fide],-1,Phase 1,,"[NaN, PIGBOS1 NT control ]",[PIGBOS1 NT control ],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
VTX-0852018,False,VTX-0852018,53,chr1:-1--1,[velia_phase1_Bona fide],-1,Phase 1,,"[HOXB-AS3 NT control , NaN]","[HOXB-AS3 NT control , HOXB-AS3 NT control , N...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
VTX-0852019,False,VTX-0852019,45,chr1:-1--1,[velia_phase2_Bonafide_Bianca],Phase 2_154,Phase 2,U9096HI220-282,"[Phase 2_154, U9096HI220-282, ZBTB37]","[Phase 2_154, Phase 4_352, sORF2154, sORF4352,...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
VTX-0852020,False,VTX-0852020,42,chr1:-1--1,[velia_phase2_Bianca_Chen],Phase 2_155,Phase 2,U9096HI220-246,"[17-242, 6xhis-HSA-HRV3C-sORF2155_1_02, Phase ...","[17-242, 57729, 6xhis-HSA-HRV3C-sORF2155_1_02,...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
VTX-0852021,False,VTX-0852021,59,chr1:-1--1,[velia_phase2_Bianca_Chen],Phase 2_156,Phase 2,U312HHH170-182,"[Phase 2_156, U312HHH170-182, ZFAND2A_1199767_...","[c7riboseqorf5, Phase 2_156, Phase 4_727, sORF...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VTX-0851963,False,VTX-0851963,79,chr1:120005395-120029951,[velia_phase3_nan],Phase 3_94,Phase 3,U738YHG260-500,"[NOTCH2/ENST00000579475.7/mRNA/5'UTR, NOTCH2/N...","[NOTCH2/ENST00000579475.7/mRNA/5'UTR, NOTCH2/N...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
VTX-0851964,False,VTX-0851964,81,chr12:6388403-6388942,[velia_phase3_nan],U738YHG260-508,Phase 3,U738YHG260-508,"[LTBR/ENST00000543542.1/ncRNA/-, U738YHG260-508]","[LTBR/ENST00000543542.1/ncRNA/-, NaN, U738YHG2...",...,4.0,2.1,XM_036165897.1,XM_036165897.1 PREDICTED: Mus musculus lymphot...,93.0,39.0,23.0,73.913,0.0,0.000129
VTX-0851965,False,VTX-0851965,81,chr4:125487463-125490005,[velia_phase3_nan],U738YHG260-510,Phase 3,U738YHG260-510,"[FAT4/NM_001291285.1/mRNA/CDS, U738YHG260-510]","[FAT4/NM_001291285.1/mRNA/CDS, NaN, U738YHG260...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
